<a href="https://colab.research.google.com/github/handsomeboyck/colab_copy/blob/master/data_prefix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
ls

 驱动器 D 中的卷是 Data
 卷的序列号是 D8DA-21A2

 D:\software_workplace\python_workplace\ssh_python_workplace\recommend\data_prefix 的目录

2021/07/30  16:59    <DIR>          .
2021/07/30  16:59    <DIR>          ..
2021/07/30  14:39    <DIR>          .ipynb_checkpoints
2021/07/30  16:59            10,481 data_prefix.ipynb
2021/07/30  14:39       265,105,635 ratings.dat
2021/07/30  16:34         1,923,952 test_set
2021/07/30  16:54         5,841,904 test_set_hash
2021/07/30  16:34       133,837,236 train_set
2021/07/30  16:54       403,964,180 train_set_hash
               6 个文件    810,683,388 字节
               3 个目录 282,565,521,408 可用字节


In [ ]:
#读取数据
data_file = './ratings.dat'
lines = open(data_file)

In [ ]:
#打印前十行
for i,line in enumerate(lines):
    if i>10:
        break;
    print(line)

1::122::5::838985046

1::185::5::838983525

1::231::5::838983392

1::292::5::838983421

1::316::5::838983392

1::329::5::838983392

1::355::5::838984474

1::356::5::838983653

1::362::5::838984885

1::364::5::838983707

1::370::5::838984596



In [ ]:
#计算数据总量
count = 0
for line in lines:
    count += 1
print(count)

10000042


In [ ]:
def read_raw_data(file_path):
    user_info = dict()
    lines = open(file_path)
    for line in lines:
        tmp = line.strip().split("::")
        if len(tmp)<4:
            continue
        ui = user_info.get(tmp[0],None)
        if ui is None:
            user_info[tmp[0]] = [(tmp[1],tmp[2],tmp[3])]
        else:
            user_info[tmp[0]].append((tmp[1],tmp[2],tmp[3]))
    return user_info

In [ ]:
user_info = read_raw_data('./ratings.dat')

In [ ]:
#简单统计下用户的行为数，基于行为数过滤掉异常用户
user_action_num = {}
for k,v in user_info.items():
    user_action_num[k] = len(v)


In [ ]:
import numpy as np
user_stat = np.asanyarray(list(user_action_num.values()))
print("用户总量：",len(user_stat))
max_num = np.max(user_stat)
min_num = np.min(user_stat)
median_num = np.median(user_stat)
avg_num = np.average(user_stat)
print(max_num)
print(min_num)
print(median_num)
print(avg_num)

用户总量： 69878
7359
20
69.0
143.10732991785684


In [ ]:
filter_user_nums = 0
for n in user_stat:
    if n >2000:
        filter_user_nums +=1
print(filter_user_nums)

97


In [ ]:
#过滤掉行为数大于2000的
def extract_valid_user(user_info):
    user_info_filter_result = {}
    for k,v in user_info.items():
        if len(v) > 2000:
            continue
        user_info_filter_result[k] = v
    return user_info_filter_result
user_info = extract_valid_user(user_info)
print(len(user_info))

69781


In [ ]:
#基于时间序列进行排序，并定义训练集和测试集
def spilt_train_test(user_info):
    train_set = []
    test_set = []
    for k,v in user_info.items():
        tmp = sorted(v,key=lambda _:_[2])
        for i in range(len(tmp)):
            if i <len(tmp)-2:
                train_set.append(str(k)+","+tmp[i][0]+","+tmp[i][1])
            else:
                test_set.append(str(k)+","+tmp[i][0]+","+tmp[i][1])
    return train_set,test_set
train_set,test_set = spilt_train_test(user_info)

In [ ]:
#将数据保存下来：
def save_data(test_data,train_data,save_path_dir):
    import random
    random.shuffle(train_data)
    random.shuffle(test_data)
    with open(save_path_dir+"train_set","w") as f:
        for line in train_data:
            f.write(line+"\n")
    with open(save_path_dir+"test_set","w") as f:
        for line in test_data:
            f.write(line+"\n")
save_path = "./"
save_data(test_set,train_set,save_path)

In [ ]:
#定义一个hash方法
def data2hash(data):
    mask60 = 0x0fffffffffffffff
    seed = 131
    hash = 0
    for s in data:
        hash = hash*seed +ord(s)
    return hash&mask60
print(data2hash("UserId=1"))
print(data2hash("MovieId=1"))
    

56860453140441778
987707537618001015


In [ ]:
def tohash(file,save_path):
    wfile = open(save_path,"w")
    with open(file) as f:
        for line in f:
            tmp = line.strip().split(",")
            user_id = data2hash("UserId="+tmp[0])
            item_id = data2hash("MovieId="+tmp[1])
            wfile.write(str(user_id)+","+str(item_id)+","+tmp[2]+"\n")
        wfile.close()
train_file_path = "./train_set"
train_file_save_path = "./train_set_hash"
test_file_path = "./test_set"
test_file_save_path = "./test_set_hash"
tohash(train_file_path,train_file_save_path)
tohash(test_file_path,test_file_save_path)

In [ ]:
ls

 驱动器 D 中的卷是 Data
 卷的序列号是 D8DA-21A2

 D:\software_workplace\python_workplace\ssh_python_workplace\recommend\data_prefix 的目录

2021/07/30  16:54    <DIR>          .
2021/07/30  16:54    <DIR>          ..
2021/07/30  14:39    <DIR>          .ipynb_checkpoints
2021/07/30  16:53             8,623 data_prefix.ipynb
2021/07/30  14:39       265,105,635 ratings.dat
2021/07/30  16:34         1,923,952 test_set
2021/07/30  16:54         5,841,904 test_set_hash
2021/07/30  16:34       133,837,236 train_set
2021/07/30  16:54       403,964,180 train_set_hash
               6 个文件    810,681,530 字节
               3 个目录 346,246,377,472 可用字节


In [ ]:
import tensorflow as tf

# 定义自己的tfrecord的格式
def get_tfrecords_example(feature,label):
    tfrecords_features = {
        "feature":tf.train.Feature(int64_list=tf.train.Int64List(value=feature)),
        "label":tf.train.Feature(float_list=tf.train.FloatList(value=label))
    }
    return tf.train.Example(
        features = tf.train.Features(feature =tfrecords_features)
    )

In [ ]:
def totfrecords(file,save_dir):
    print("Process to tfrecord File:%s ...." % file)
    num = 0
    writer = tf.io.TFRecordWriter(save_dir+"/"+"part-0000"+str(num)+".tfrecords")
    lines = open(file)
    for i,line in enumerate(lines):
        tmp = line.strip().split(",")
        feature = [int(tmp[0]),int(tmp[1])]
        label =[float(1) if float(tmp[2])>=3 else float(0)]
        example = get_tfrecords_example(feature,label)
        writer.write(example.SerializeToString())
        if (i+1) % 200000 ==0:
            writer.close()
            num +=1
            writer = tf.io.TFRecordWriter(save_dir+"/"+"part-0000"+str(num)+".tfrecords")
            
    print("Process To tfrecord File: %s End" %file)
    writer.close()

import os

train_file_path ="./train_set_hash"
train_totfrecord ="./train_tf"
test_file_path = "./test_set_hash"
test_totfrecord = "./test_tf"
totfrecords(train_file_path,train_totfrecord)
totfrecords(test_file_path,test_totfrecord)


            

Process to tfrecord File:./train_set_hash ....
Process To tfrecord File: ./train_set_hash End
Process to tfrecord File:./test_set_hash ....
Process To tfrecord File: ./test_set_hash End
